In [144]:
class GDregressor:
    def __init__(self, learning_rate=0.01, epochs=100):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        y_train = np.array(y_train).reshape(-1,1)

        m, n = X_train.shape

        # initialize
        self.intercept_ = 0.0
        self.coef_ = np.ones((n,1))

        for _ in range(self.epochs):

            # prediction
            y_hat = (X_train @ self.coef_) + self.intercept_

            # error
            error = y_train - y_hat

            # gradients
            d_b = -2 * np.mean(error)
            d_w = -2 * (X_train.T @ error) / m

            # update
            self.intercept_ -= self.learning_rate * d_b
            self.coef_ -= self.learning_rate * d_w

            loss = np.mean(error**2)

            if _ % 300 == 0:
                print("Epoch:", _, "Loss:", loss)


    def predict(self, X_test):
        X_test = np.array(X_test)
        return X_test @ self.coef_ + self.intercept_

    def mse(self, y_true, y_pred):
        return np.mean((y_true - y_pred)**2)

    def r2_score(self,y_true, y_pred):
        y_true = np.array(y_true).reshape(-1,1)
        y_pred = np.array(y_pred).reshape(-1,1)

        ss_res = np.sum((y_true - y_pred) ** 2)   # residual sum of squares
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)  # total sum of squares

        r2 = 1 - (ss_res / ss_tot)
        return r2



In [145]:
import numpy as np

# function to generate a raw data
def generate_lr_data(
    n_samples=200,
    n_features=1,
    noise_std=1.0,
    weight_range=(-5, 5),
    bias_range=(-3, 3),
    x_range=(-10, 10),
    add_outliers=False,
    outlier_ratio=0.05,
    outlier_strength=15,
    seed=42
):
    """
    Generate synthetic Linear Regression data:
        y = Xw + b + noise

    Returns:
        X : (n_samples, n_features)
        y : (n_samples, 1)
        true_w : (n_features, 1)
        true_b : float
    """
    rng = np.random.default_rng(seed)

    # Features
    X = rng.uniform(x_range[0], x_range[1], size=(n_samples, n_features))

    # True weights and bias
    true_w = rng.uniform(weight_range[0], weight_range[1], size=(n_features, 1))
    true_b = rng.uniform(bias_range[0], bias_range[1])

    # Perfect line/plane
    y_clean = X @ true_w + true_b

    # Add Gaussian noise
    noise = rng.normal(0, noise_std, size=(n_samples, 1))
    y = y_clean + noise

    # Optionally add outliers
    if add_outliers:
        n_outliers = int(n_samples * outlier_ratio)
        outlier_indices = rng.choice(n_samples, n_outliers, replace=False)

        # Make y much bigger/smaller randomly
        y[outlier_indices] += rng.normal(0, outlier_strength, size=(n_outliers, 1))

    return X, y, true_w, true_b


def train_test_split_manual(X, Y, test_size=0.2, random_state=42):
    rng = np.random.default_rng(random_state)

    X = np.array(X)
    Y = np.array(Y).reshape(-1, 1)

    n_samples = X.shape[0]

    indices = np.arange(n_samples)
    rng.shuffle(indices)

    test_count = int(n_samples * test_size)

    test_idx = indices[:test_count]
    train_idx = indices[test_count:]

    return X[train_idx], X[test_idx], Y[train_idx], Y[test_idx]


In [146]:
X, y, w_true, b_true = generate_lr_data(
    n_samples=300,
    n_features=10,
    noise_std=1.5
)

print("True weights:", w_true.ravel())
print("True bias:", b_true)


True weights: [ 3.29221141  0.94095987 -2.55145253  2.45675001 -4.15519104  2.74417834
  0.89985222 -3.89249682 -4.20494437  2.50836333]
True bias: -1.242384560976977


In [147]:
X_train, X_test, y_train, y_test = train_test_split_manual(X, y, test_size=0.2, random_state=2)

"""
    During training, gradient descent initially diverged due to large feature magnitudes.
    This was resolved by applying z-score normalization (mean=0, std=1) to input features, which significantly improved convergence stability.
"""
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std



gdr = GDregressor(learning_rate=0.001, epochs=1000)
gdr.fit(X_train, y_train)

print("True:", w_true.ravel(), b_true)
print("Learned:", gdr.coef_.ravel(), gdr.intercept_)

"""
    My gradient descent converged successfully as indicated by decreasing loss; 
    coefficient values differ from ground truth due to feature standardization.
"""

Epoch: 0 Loss: 2950.5457335678684
Epoch: 300 Loss: 895.9227510035776
Epoch: 600 Loss: 284.0153149452275
Epoch: 900 Loss: 94.10280852559185
True: [ 3.29221141  0.94095987 -2.55145253  2.45675001 -4.15519104  2.74417834
  0.89985222 -3.89249682 -4.20494437  2.50836333] -1.242384560976977
Learned: [ 18.46099573   4.97743551 -13.39405846  11.64592431 -20.81560901
  12.28338385   3.84527283 -17.71291302 -19.9999052   12.57180844] 3.082166908407099


'\n    My gradient descent converged successfully as indicated by decreasing loss; \n    coefficient values differ from ground truth due to feature standardization.\n'

In [148]:
y_pred = gdr.predict(X_test)

print("Predicted:", y_pred.ravel())

Predicted: [  7.87507253  43.65022759 116.33122892 -97.61044568  10.98960873
 -66.55188456  -6.00581183 -75.21054312 -64.08465188  21.85072646
  -1.3742951   18.90881861 -20.23757512 -39.14022115  14.44685565
  55.04129086 -13.53423353  35.66211641  56.11425419  33.71423522
  10.28499852  36.58364661  41.83397078 -55.51472599 -41.52125557
  -7.10941503 -48.74700215 -47.45037428 -35.98452663  -0.73011202
  62.33264493 -35.53188691 -42.57024925  94.50073939  59.14680544
 -77.64666077  -2.68067512  22.84375457 -23.00238024  60.35826543
 -12.10265045  60.0931546   30.15824817  52.50697398 -44.15710453
 -58.57910068  43.52024377 -43.67416157  53.42677804 107.11559797
 -77.06189641 -33.91890756  39.61278986  -1.90167891  22.48187406
  16.29033505  -4.52981196  26.96054541  25.94135286 -91.45381713]


In [149]:
mse = np.mean((y_test - y_pred)**2)
r2 = gdr.r2_score(y_test, y_pred)

print("MSE:", mse)
print("R2 Score:", r2)

MSE: 77.95026515026714
R2 Score: 0.9762891612287307
